In [6]:
import sys
import re
from pathlib import Path
import xml.etree.ElementTree as ET

# ============================================
# Konfiguration
# ============================================

# Eingabedatei: TwinCAT Export
XML_PATH = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml")

# PLCreX-Ausgabe (Verzeichnis!)
PLC_REX_EXPORT_DIR = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\plcrex_exports")
PLC_REX_BASENAME = "export_fbd"  # nur für unseren ZIEL-Dateinamen, nicht für den PLCreX-Aufruf

PLC_REX_USE_FORMAL = True   # True: TON(IN := ..., PT := ...); False: TON(..., ...)
PLC_REX_USE_BWD = False     # False = forward translation, True = backward


# ============================================
# Helpers für XML
# ============================================

def load_export_root(xml_path: Path) -> ET.Element:
    if not xml_path.exists():
        raise FileNotFoundError(f"export.xml nicht gefunden: {xml_path}")
    text = xml_path.read_text(encoding="utf-8", errors="ignore")
    # Default-Namespace entfernen
    text_wo_ns = re.sub(r'\sxmlns="[^"]+"', '', text, count=1)
    return ET.fromstring(text_wo_ns)

def detect_pou_lang(root: ET.Element):
    """Liefert dict: POU-Name -> 'FBD', 'ST' oder 'OTHER'."""
    pou_lang = {}
    pou_by_name = {}
    for pou in root.findall(".//pou"):
        name = pou.get("name")
        pou_by_name[name] = pou
        body = pou.find("body")
        if body is None:
            pou_lang[name] = "OTHER"
        elif body.find("FBD") is not None:
            pou_lang[name] = "FBD"
        elif body.find("ST") is not None:
            pou_lang[name] = "ST"
        else:
            pou_lang[name] = "OTHER"
    return pou_lang, pou_by_name

def split_st_by_pou(st_text: str):
    """
    Versucht, den von PLCreX erzeugten ST-Code in einzelne POUs zu splitten.
    Heuristik:
      - Beginn eines Blocks: PROGRAM|FUNCTION_BLOCK|FUNCTION <Name>
      - Alles bis zum nächsten Block gehört dazu.
    Falls PLCreX keine solchen Header schreibt, landet alles unter '__all__'.
    """
    blocks = {}
    pattern = re.compile(
        r'^\s*(PROGRAM|FUNCTION_BLOCK|FUNCTION)\s+([A-Za-z_]\w*)',
        re.IGNORECASE | re.MULTILINE,
    )

    matches = list(pattern.finditer(st_text))
    if not matches:
        blocks["__all__"] = st_text.strip()
        return blocks

    for i, m in enumerate(matches):
        start = m.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(st_text)
        name = m.group(2)
        block = st_text[start:end].strip()
        blocks[name] = block

    return blocks


# ============================================
# 1) TwinCAT-Export einlesen und POU-Sprachen anzeigen
# ============================================

root = load_export_root(XML_PATH)
pou_lang, pou_by_name = detect_pou_lang(root)

print("Gefundene POUs nach Sprache:")
lang_stats = {}
for name, lang in pou_lang.items():
    lang_stats.setdefault(lang, 0)
    lang_stats[lang] += 1
print(lang_stats)

print("\nBeispiel-POUs (erste 10):")
for i, (name, lang) in enumerate(pou_lang.items()):
    print(f"  {name}: {lang}")
    if i >= 9:
        break


# ============================================
# 2) PLCreX-Aufruf im Notebook (ohne subprocess.run)
#    -> EXPORT ist ein VERZEICHNIS
# ============================================

PLC_REX_EXPORT_DIR.mkdir(parents=True, exist_ok=True)

formal_flag = "--formal" if PLC_REX_USE_FORMAL else "--no-formal"
bwd_flag = "--bwd" if PLC_REX_USE_BWD else "--no-bwd"

# WICHTIG: nur ZWEI Positionsargumente: SOURCE und EXPORT (Verzeichnis!)
cmd = (
    f'"{sys.executable}" -m plcrex fbd-to-st '
    f'{formal_flag} {bwd_flag} '
    f'"{XML_PATH}" "{PLC_REX_EXPORT_DIR}"'
)

print("\nStarte PLCreX (Notebook-Aufruf):")
print(cmd)
print("-" * 50)

result = get_ipython().system(cmd)  # Startet plcrex im gleichen venv

print("\nRückgabecode von PLCreX:", result)

# ============================================
# 2b) Generierte .st-Datei(en) suchen
# ============================================

# Rekursiv alle .st-Dateien unterhalb von PLC_REX_EXPORT_DIR suchen
st_candidates = sorted(
    PLC_REX_EXPORT_DIR.rglob("*.st"),
    key=lambda p: p.stat().st_mtime,
    reverse=True,
)

if not st_candidates:
    raise FileNotFoundError(
        f"Keine .st-Dateien unter {PLC_REX_EXPORT_DIR} gefunden. "
        "PLCreX hat vermutlich nichts exportiert."
    )

print("\nGefundene .st-Dateien (neueste zuerst):")
for p in st_candidates[:5]:
    print(" ", p)

actual_st = st_candidates[0]
print(f"\nVerwende ST-Datei: {actual_st}")

# Stabiler Zielname (z.B. export_fbd.st im Haupt-Export-Ordner)
st_path = PLC_REX_EXPORT_DIR / f"{PLC_REX_BASENAME}.st"
if actual_st.resolve() != st_path.resolve():
    st_text_tmp = actual_st.read_text(encoding="utf-8", errors="ignore")
    st_path.write_text(st_text_tmp, encoding="utf-8")
    print(f"ST-Datei nach {st_path} kopiert (für weitere Verarbeitung).")

print("Endgültig verwendete ST-Datei:", st_path)


# ============================================
# 3) ST-Datei einlesen und nach POU splitten
# ============================================

st_text = st_path.read_text(encoding="utf-8", errors="ignore")
if "END_unknown" in st_text and "PROGRAM" not in st_text.upper():
    print("\n*** WARNUNG ***")
    print("PLCreX hat offenbar keine validen FBD-Netzwerke gefunden.")
    print("Das deutet stark darauf hin, dass die Quelle kein reines PLCopen-XML ist,")
    print("oder dass TwinCAT-spezifische Erweiterungen vorkommen, die PLCreX nicht versteht.")
    print("-> Bitte POU explizit als PLCopen XML neu exportieren und diese Datei als SOURCE verwenden.\n")
else:
    print("\nST-Code sieht plausibel aus, weiter mit Splitten nach POU ...")
plcrex_st_blocks = split_st_by_pou(st_text)

print("\nGefundene ST-Blöcke (POUs) in der PLCreX-Ausgabe:")
for i, name in enumerate(plcrex_st_blocks.keys()):
    print(" ", name)
    if i >= 9:
        break
if len(plcrex_st_blocks) > 10:
    print("  ...")


# ============================================
# 4) Beispiel-Ausgabe: ST-Code eines konkreten FBD-POUs anzeigen
# ============================================

example_fbd_pou = None
for name, lang in pou_lang.items():
    if lang == "FBD":
        example_fbd_pou = name
        break

if example_fbd_pou:
    print(f"\n=== Beispiel ST-Code für FBD-POU '{example_fbd_pou}' ===\n")
    st_block = plcrex_st_blocks.get(example_fbd_pou) or plcrex_st_blocks.get("__all__")
    if st_block:
        print(st_block[:3000])  # nur die ersten 3000 Zeichen anzeigen
    else:
        print(f"Kein ST-Block für {example_fbd_pou} in der PLCreX-Ausgabe gefunden.")
else:
    print("\nKein FBD-POU gefunden, der mit PLCreX übersetzt werden könnte.")


Gefundene POUs nach Sprache:
{'FBD': 49, 'ST': 1}

Beispiel-POUs (erste 10):
  HRL_SkillSet: FBD
  MBS_SkillSet: FBD
  SST_SkillSet: FBD
  VSG_SkillSet: FBD
  AutomaticColorDetection_nichtfertig: FBD
  AxisControl_MultipleInputSensors: FBD
  AxisControl_Encoder: FBD
  CompressorControl: FBD
  SecuringWorkpiece: FBD
  StampingProcess: FBD

Starte PLCreX (Notebook-Aufruf):
"d:\MA_Python_Agent\plcrex_venv39\Scripts\python.exe" -m plcrex fbd-to-st --formal --no-bwd "C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml" "C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\plcrex_exports"
--------------------------------------------------
--------------------------------------------------
Tool:		FBD-to-ST-Compiler
Version:	2.0.0
Author:		Marcel C. Werner
Start:		09:08:17 2025-11-29
--------------------------------------------------
//--- This file was generated by PLCreX ---
//--- https://github.com/marwern/PLCreX ---
//--------------------------------

d:\MA_Python_Agent\plcrex_venv39\lib\site-packages\z3\z3core.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
